In [1]:

import pandas as pd

from tqdm.notebook import tqdm
import ast
tqdm.pandas()


save = True

In [ ]:
match_data_df = pd.read_pickle('match_data_version1.pickle')

if save :match_data_df.to_csv("intermediate1.csv")

In [ ]:
indexList = [i for i in match_data_df[match_data_df.gameMode != 'CLASSIC'].index if i != 0]
match_data_df.drop(indexList, inplace=True)

indexList2 = [i for i in match_data_df[match_data_df.gameDuration <= 300].index if i != 0]
match_data_df.drop(indexList2, inplace=True)

if save :match_data_df.to_csv("intermediate2.csv")

In [ ]:
match_data_df.drop(['gameId',  'gameType', 'mapId', 'gameMode', 'participantIdentities'], axis=1, inplace=True)
if save :match_data_df.to_csv("intermediate3.csv")

In [ ]:
match_data_df = match_data_df.join(pd.json_normalize(match_data_df.pop('participants')))
if save :match_data_df.to_csv("intermediate4.csv")

In [ ]:
match_data_df['zawodnicy'] = match_data_df[[i for i in range(10)]].apply(list, axis=1)
if save :match_data_df.to_csv("intermediate5.csv")

In [ ]:
match_data_df.drop(columns=[i for i in range(10)], inplace=True)
if save :match_data_df.to_csv("intermediate6.csv")

In [ ]:
# Custom explode function with progress bar
def custom_explode_with_progress(df, column):
    rows = []
    total_rows = len(df)
    with tqdm(total=total_rows) as pbar:
        for _, row in df.iterrows():
            for val in row[column]:
                new_row = row.to_dict()
                new_row[column] = val
                rows.append(new_row)
            pbar.update(1)  # Update progress bar
    return pd.DataFrame(rows)

# Explode the DataFrame with progress bar
match_data_df = custom_explode_with_progress(match_data_df, 'zawodnicy')
if save :match_data_df.to_csv("intermediate7.csv")

In [3]:
match_data_df = match_data_df[['gameCreation', 'gameDuration', 'gameVersion', 'zawodnicy']]
if save :match_data_df.to_csv("intermediate8.csv")

In [4]:
match_data_df['zawodnicy'] = match_data_df['zawodnicy'].str.replace("\'","\"")
if save :match_data_df.to_csv("intermediate9.csv")

In [5]:
match_data_df.drop(match_data_df[match_data_df.isna().any(axis=1)].index,inplace=True)
if save :match_data_df.to_csv("intermediate10.csv")

In [6]:
match_data_df['zawodnicy'] = match_data_df['zawodnicy'].apply(ast.literal_eval)
if save :match_data_df.to_csv("intermediate11.csv")

In [7]:
match_data_df = match_data_df.join(pd.json_normalize(match_data_df.pop('zawodnicy')))
if save :match_data_df.to_csv("intermediate12.csv")

In [18]:
match_data_df = match_data_df[['gameCreation', 'gameDuration', 'gameVersion', 'stats.win', 'stats.kills', 'stats.deaths', 'stats.assists', 'stats.firstBloodKill', 'stats.visionScore', 'stats.pentaKills', 'stats.goldEarned', 'stats.damageDealtToTurrets', 'stats.totalDamageDealtToChampions']]
if save :match_data_df.to_csv("intermediate13.csv")

In [19]:

def categorize_game_duration(duration):
    if duration < 20 * 60:
        return "Krótka"
    elif 20 * 60 <= duration <= 30 * 60:
        return "Normalna"
    else:
        return "Długa"
match_data_df['gameDurationDiscrete'] = match_data_df['gameDuration'].apply(categorize_game_duration)
if save :match_data_df.to_csv("intermediate14.csv")

In [22]:
def game_duration_to_minutes(duration):
    return duration // 60
match_data_df['gameDurationMinutes'] = match_data_df['gameDuration'].apply(game_duration_to_minutes)
if save :match_data_df.to_csv("intermediate15.csv")

In [25]:
def discretize_kda(row):
    kills = row['stats.kills']
    deaths = row['stats.deaths']
    assists = row['stats.assists']
    if deaths == 0:
        return "Duże"
    kda = (kills+assists)/deaths
    if kda < 1:
        return 'Małe'
    if kda < 2:
        return 'Średnie'
    return 'Duże'
match_data_df['kdaDiscrete'] = match_data_df.apply(discretize_kda, axis=1)
if save : match_data_df.to_csv("intermediate16.csv")

In [29]:
match_data_df['gameCreationMonth'] = pd.to_datetime(match_data_df['gameCreation'], unit='ms').dt.month
if save: match_data_df.to_csv("intermediate17.csv")

In [32]:
match_data_df['isWeekend'] = pd.to_datetime(match_data_df['gameCreation'], unit='ms').dt.dayofweek>=5
if save: match_data_df.to_csv("intermediate18.csv")

In [36]:
match_data_df['gameVersion'] = match_data_df['gameVersion'].str.split('.').str[:2].str.join('.')
if save: match_data_df.to_csv("intermediate19.csv")

In [48]:
indexList3 = [i for i in match_data_df[match_data_df['gameVersion'].str[:2] != '10'].index if i != 0]

match_data_df.drop(indexList3, inplace=True)
if save: match_data_df.to_csv("intermediate20.csv")

In [50]:
match_data_df.drop(['stats.kills', 'stats.assists', 'gameCreation', 'gameDuration'], axis=1, inplace=True)
if save: match_data_df.to_csv("intermediate21.csv")

In [76]:
match_data_df.dropna(axis=0,inplace = True)
if save: match_data_df.to_csv("intermediate22.csv")

In [79]:
unique_combinations = match_data_df[['stats.win', 'kdaDiscrete', 'stats.firstBloodKill']].drop_duplicates().reset_index(drop=True)

match_data_df['profilStatystykGraczaWMeczu'] = match_data_df[['stats.win', 'kdaDiscrete', 'stats.firstBloodKill']].apply(
    lambda row: unique_combinations[
        (unique_combinations['stats.win'] == row['stats.win']) &
        (unique_combinations['kdaDiscrete'] == row['kdaDiscrete']) &
        (unique_combinations['stats.firstBloodKill'] == row['stats.firstBloodKill'])
    ].index[0], axis=1
)
match_data_df.drop(['stats.win', 'kdaDiscrete', 'stats.firstBloodKill'], axis=1, inplace=True)
if save: 
    match_data_df.to_csv("intermediate23.csv")
    unique_combinations.to_csv("dimprofilstatystyk23.csv")

In [94]:
unique_combinations2 = match_data_df[['gameDurationDiscrete', 'gameDurationMinutes']].drop_duplicates().reset_index(drop=True)

match_data_df['gameDurationDim'] = match_data_df[['gameDurationDiscrete', 'gameDurationMinutes']].apply(
    lambda row: unique_combinations2[
        (unique_combinations2['gameDurationDiscrete'] == row['gameDurationDiscrete']) &
        (unique_combinations2['gameDurationMinutes'] == row['gameDurationMinutes'])
    ].index[0], axis=1
)
match_data_df.drop(['gameDurationMinutes', 'gameDurationDiscrete'], axis=1, inplace=True)
if save: 
    match_data_df.to_csv("intermediate24.csv")
    unique_combinations.to_csv("dimprofilstatystyk24.csv")
    unique_combinations2.to_csv("dimgameduration24.csv")

In [95]:
unique_combinations3 = match_data_df[['gameVersion', 'gameCreationMonth', 'isWeekend']].drop_duplicates().reset_index(drop=True)

match_data_df['gameVersionDim'] = match_data_df[['gameVersion', 'gameCreationMonth', 'isWeekend']].apply(
    lambda row: unique_combinations3[
        (unique_combinations3['gameVersion'] == row['gameVersion']) &
        (unique_combinations3['gameCreationMonth'] == row['gameCreationMonth'])&
        (unique_combinations3['isWeekend'] == row['isWeekend'])
    ].index[0], axis=1
)
match_data_df.drop(['gameVersion', 'gameCreationMonth', 'isWeekend'], axis=1, inplace=True)
if save: 
    match_data_df.to_csv("intermediate25.csv")
    unique_combinations.to_csv("dimprofilstatystyk25.csv")
    unique_combinations2.to_csv("dimgameduration25.csv")
    unique_combinations3.to_csv("dimgamecreationtime25.csv")

,Unnamed: 0,stats.deaths,stats.visionScore,stats.pentaKills,stats.goldEarned,stats.damageDealtToTurrets,stats.totalDamageDealtToChampions,profilStatystykGraczaWMeczu,gameDurationDim,gameVersionDim
0,0,6.0,14.0,0.0,6844.0,1008.0,10844.0,0,0,0
1,1,6.0,34.0,0.0,5205.0,427.0,7095.0,1,0,0
2,2,5.0,8.0,0.0,8226.0,515.0,13458.0,1,0,0
3,3,7.0,21.0,0.0,7911.0,451.0,9670.0,1,0,0
4,4,7.0,14.0,0.0,8815.0,1103.0,14972.0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
876299,982312,6.0,10.0,0.0,7182.0,1168.0,13231.0,1,15,7
876300,982313,3.0,26.0,0.0,5796.0,26.0,6479.0,1,15,7
876301,982314,4.0,6.0,0.0,7741.0,2034.0,10619.0,1,15,7
876302,982315,3.0,30.0,0.0,6531.0,820.0,3574.0,5,15,7
